# compas_fea2 simple script approach

This example covers the analysis of a simple 2D frame structure generated completely from scratch. 

### NOTE: This example is in SI (m, kg, s)

In [1]:
import compas_fea2

from compas_fea2.model import Model, elements
from compas_fea2.model import Part
from compas_fea2.model import Node
from compas_fea2.model import ElasticIsotropic, Steel
from compas_fea2.model import CircularSection, RectangularSection, MembraneSection
from compas_fea2.model import ShellSection
from compas_fea2.model import BeamElement, MembraneElement, ShellElement
from compas_fea2.model import RollerBCXY
from compas_fea2.model import FixedBC, PinnedBC
from compas_fea2.model import NodesGroup
from compas_fea2.model import BeamEndPinRelease

from compas_fea2.problem import Problem
from compas_fea2.problem import PointLoad
from compas_fea2.problem import GravityLoad
from compas_fea2.problem import FieldOutput
from compas_fea2.problem import StaticStep, ModalAnalysis
from compas_fea2.problem import GeneralDisplacement

from compas_fea2.results import Results
from pathlib import Path
from pprint import pprint

from compas_fea2 import TEMP

## Plug-in registration

compas_fea2 implements a plug-in architecture. The compas_fea2 main package only defines the general API for a Finite Element Analysis, while the actual implementation in the a specific backend is done in the corresponding plug-in, whcih must registered and the beginning of the script. 

In [2]:
compas_fea2.set_backend('abaqus')

Abaqus implementations registered...
Opensees implementations registered...
Ansys implementations registered...


### Verbosity
you can choose how much info you want to see by setting the `VERBOSE` property
to `True`

In [3]:
compas_fea2.config.VERBOSE = True

## Model

The fist step (of an FEA) is the generation of the analysis `model`. 
A model is always composed by:

1) `Parts`:
2) `Boundary Conditions`:

it can als include:

3) `Constraints` and `Interactions`:
4) `Groups of Parts`:


In [4]:
model = Model()

### Parts

`Parts` are subregions of the model that can be considered to be independent from each other. However, similar parts of a model can still be modelled as different `Parts`. 

A good example of a `Part` could be a brick. 

In [5]:
frame = model.add_part(Part())
print(frame)

AbaqusPart(1492754614016) registered to AbaqusModel(1492754614112).

AbaqusPart
----------
name : AP_1492754614016

number of elements : 0
number of nodes    : 0
number of groups   : 0



Nodes are added to the `Part`. In this example, we want to create a simple  frame with 4 nodes at the base and 1 node at height of 5m.

In [6]:
coordinates = [[0., 0., 5.], [5., -5., 0.], [5., 5., 0.], [-5., 5., 0.], [-5., -5., 0.]]
nodes = [Node(xyz=node) for node in coordinates]

print(nodes[0])

AbaqusNode(1492754614736)


We can now add the material properties we plan to use in our analysis. These material will then be available in the 'Model' and can be used in the definition of the structural sections.

In [7]:
mat = ElasticIsotropic(E=10*10**9, v=0.3, density=1000)

`compas_fea2` uses the same convention as Abaqus where the materials are assigned to the section and not to the elements (sections are assigned to the elements).
The structural properties of the section are computed automatically accordingly to the type of section chosen (`RectangularSection` in this case)

In [8]:
frame_sec = RectangularSection(w=0.05, h=0.1, material=mat)

Structural elements are defined by their type, connectivity and associated section. In this case we use a `BeamElement`, so we need to specify the 2 nodes it is connected to, with the 'BoxSection' defined before.
Elements can be added to the `Part` one by one or as a list. 

In [9]:
beams = [frame.add_element(BeamElement(nodes=[nodes[0], node], section=frame_sec)) for node in nodes[1:]]

Node AbaqusNode(1492754614736) registered to AbaqusPart(1492754614016).
Node AbaqusNode(1492754613200) registered to AbaqusPart(1492754614016).
Element AbaqusBeamElement(1492660302752) registered to AbaqusPart(1492754614016).
SKIPPED: Node AbaqusNode(1492754614736) already in part.
Node AbaqusNode(1492754616128) registered to AbaqusPart(1492754614016).
SKIPPED: Section AbaqusRectangularSection(1492754615552) already in part.
Element AbaqusBeamElement(1492754614832) registered to AbaqusPart(1492754614016).
SKIPPED: Node AbaqusNode(1492754614736) already in part.
Node AbaqusNode(1492754613920) registered to AbaqusPart(1492754614016).
SKIPPED: Section AbaqusRectangularSection(1492754615552) already in part.
Element AbaqusBeamElement(1492660297840) registered to AbaqusPart(1492754614016).
SKIPPED: Node AbaqusNode(1492754614736) already in part.
Node AbaqusNode(1492660303136) registered to AbaqusPart(1492754614016).
SKIPPED: Section AbaqusRectangularSection(1492754615552) already in part.
E

## Initial Boundary Conditions

In [10]:
model.add_pin_bc(node=nodes[1])
model.add_bcs(bc=FixedBC(), nodes=nodes[2:])

[AbaqusFixedBC(1492754613824),
 AbaqusFixedBC(1492754613824),
 AbaqusFixedBC(1492754613824)]

In [11]:
model.summary()


++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
compas_fea2 Model: AM_1492754614112
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

description: N/A
author: N/A

Parts
-----
AP_1492754614016
    # of nodes: 5
    # of elements: 4

Interactions
------------
N/A

Constraints
-----------
N/A

Boundary Conditions
-------------------
AP_1492754614016: 
  AbaqusPinnedBC(1492754616176) - [AbaqusNode(1492754613200)]
  AbaqusFixedBC(1492754613824) - [AbaqusNode(1492754613920), AbaqusNode(1492754616128), AbaqusNode(1492660303136)]



'\n++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\ncompas_fea2 Model: AM_1492754614112\n++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n\ndescription: N/A\nauthor: N/A\n\nParts\n-----\nAP_1492754614016\n    # of nodes: 5\n    # of elements: 4\n\nInteractions\n------------\nN/A\n\nConstraints\n-----------\nN/A\n\nBoundary Conditions\n-------------------\nAP_1492754614016: \n  AbaqusPinnedBC(1492754616176) - [AbaqusNode(1492754613200)]\n  AbaqusFixedBC(1492754613824) - [AbaqusNode(1492754613920), AbaqusNode(1492754616128), AbaqusNode(1492660303136)]\n'

## Problem definition
The second part of the FEA is to define the `Problem` to be solved. To define a `Problem` you need a `Model` and the `Steps` in which the loads are applied.  

In [12]:

problem = Problem(model=model, name='test')

In [13]:
step_0 = problem.add_step(StaticStep())
step_0.add_gravity_load()

step_1 = problem.add_step(StaticStep())
step_1.add_point_load(x=1000, z=-1000, node=nodes[0])
# # Define the field outputs required
fout = step_1.add_output(FieldOutput())

Once the `Probelm` is set, you can solve it running the analysis. You need to specify the location of the analysis files generated by Abaqus.
The output log for now is limited to the one automatically generated by Abaqus, but the plan is to expand it to have more info about the status of the analysis, possible warnings or errors, etc. 

In [14]:
# # Solve the problem
problem.analyse(path=Path(TEMP).joinpath(problem.name))

SKIPPED: Element AbaqusBeamElement(1492660302752) already in part.
SKIPPED: Element AbaqusBeamElement(1492754614832) already in part.
SKIPPED: Element AbaqusBeamElement(1492754686928) already in part.
SKIPPED: Element AbaqusBeamElement(1492660297840) already in part.
Model generated in  0.0014 secs
Problem generated in  0.0002 secs
Input file generated in: c:\code\myrepos\from_compas\fea2\temp\test\test.inp
Finished writing input file in 0.0115 secs
Analysis initiated from SIMULIA established products
Abaqus JOB test
Abaqus 2021
Begin Analysis Input File Processor
5/4/2022 3:55:53 PM
Run pre.exe
5/4/2022 3:55:59 PM
End Analysis Input File Processor
Begin Abaqus/Standard Analysis
5/4/2022 3:55:59 PM
Run standard.exe
5/4/2022 3:56:06 PM
End Abaqus/Standard Analysis
Begin SIM Wrap-up
5/4/2022 3:56:08 PM
Run SMASimUtility.exe
5/4/2022 3:56:08 PM
End SIM Wrap-up
Abaqus JOB test COMPLETED

Abaqus License Manager checked out the following licenses:
Abaqus/Standard checked out 5 tokens from Fl

## Results Visualization

...not there yet... :)

In [16]:
##### --------------------- POSTPROCESS RESULTS -------------------------- #####
results = Results.from_problem(problem)
pprint(results.nodal)


Abaqus License Manager checked out the following license:
"cae_teaching" from Flexnet server lic-abaqus.ethz.ch
<37 out of 40 licenses remain available>.

Data stored successfully in c:\code\myrepos\from_compas\fea2\temp\test\test-results.pkl
Data stored successfully in c:\code\myrepos\from_compas\fea2\temp\test\test-info.pkl
{'ASS_1492660301936': {'cfm': {0: 1414.213623046875,
                               1: 0.0,
                               2: 0.0,
                               3: 0.0,
                               4: 0.0},
                       'cfx': {0: 1000.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0},
                       'cfy': {0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0},
                       'cfz': {0: -1000.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0},
                       'cmm': {0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0},
                       'cmx': {0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0},
                       'cmy': {0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0},
                      